In [1]:
import os
import sys
import datetime

import pickle
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))
from SRModels.deep_learning_models.EDSR_model import EDSR
from SRModels.loading_methods import load_dataset_as_patches
from SRModels.constants import EDSR_PATCH_SIZE, EDSR_STRIDE, RANDOM_SEED, EDSR_SCALE_FACTOR

In [2]:
HR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/HR"))
LR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/LR"))

In [3]:
X, Y = load_dataset_as_patches(HR_ROOT, LR_ROOT, mode="scale", patch_size=EDSR_PATCH_SIZE, stride=EDSR_STRIDE, scale_factor=EDSR_SCALE_FACTOR)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}, Y_val shape: {Y_val.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

X_train shape: (3673, 96, 96, 3), Y_train shape: (3673, 192, 192, 3)
X_val shape: (409, 96, 96, 3), Y_val shape: (409, 192, 192, 3)
X_test shape: (454, 96, 96, 3), Y_test shape: (454, 192, 192, 3)


In [4]:
model = EDSR()

model.setup_model(scale_factor=EDSR_SCALE_FACTOR, num_res_blocks=16, num_filters=64)

Model: "EDSR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   1792        ['input[0][0]']                  
                                64)                                                               
                                                                                                  
 separable_conv2d (SeparableCon  (None, None, None,   4736       ['conv2d[0][0]']                 
 v2D)                           64)                                                            

In [5]:
# Train EDSR and capture callbacks for metrics
time_cb, mem_cb = model.fit(
    X_train, Y_train, X_val, Y_val, 
    batch_size=16, 
    epochs=300, 
    use_augmentation=True, 
    use_mix=True
)

Training on GPU: /physical_device:GPU:0
Epoch 1/100
  7/230 [..............................] - ETA: 1:30 - loss: 1.4936 - psnr: -6.9486 - ssim: 7.2204e-04

KeyboardInterrupt: 

In [6]:
# Evaluate and prepare metrics dictionary
results = model.evaluate(X_test, Y_test)
metrics_dict = {
    "eval_loss": float(results[0]),
    "eval_psnr": float(results[1]),
    "eval_ssim": float(results[2]),
    "epoch_time_sec": time_cb.mean_time_value(),
    "memory": mem_cb.as_dict()
}
print(metrics_dict)

2/2 [==============================] - 0s 70ms/step - loss: nan - psnr: -9.5557 - ssim: 1.8082e-04
Loss: nan, PSNR: -9.56 dB, SSIM: 0.0002
{'eval_loss': nan, 'eval_psnr': -9.555675506591797, 'eval_ssim': 0.00018081719463225454, 'epoch_time_sec': 21.72439379998832, 'memory': {'gpu_mean_current_mb': 15.3887939453125, 'gpu_peak_mb': 2536.234619140625}}


In [7]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

model.save(directory=f"models/EDSR/EDSR_{timestamp}", timestamp=timestamp)

Model saved to models/EDSR/EDSR_20250908_041338\EDSR_x2_20250908_041338.h5


In [8]:
# Save evaluation/time/memory metrics next to the model
metrics_path = os.path.abspath(os.path.join(os.getcwd(), f"models/EDSR/EDSR_{timestamp}/EDSR_{timestamp}_metrics.pkl"))

with open(metrics_path, "wb") as f:
    pickle.dump(metrics_dict, f)
    
print(f"Saved metrics to {metrics_path}")

Saved metrics to c:\Users\bgmanuel\InteligenciaArtificial\MasterInteligenciaArtificial\Periodo2\TFM\Super-Resolution-Images-for-3D-Printing-Defect-Detection\SRModels\deep_learning_models\models\EDSR\EDSR_20250908_041338\EDSR_20250908_041338_metrics.pkl
